In [126]:
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score 

Given Dataset

In [127]:
train = pd.read_csv('Titanic-Dataset.csv')
train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [128]:
test = train.drop('Survived', axis =1)
test.head(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [129]:
#Detecting Missing values in the Dataset
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [130]:
# Replace missing values in the 'Embarked' column with 'S'
train = train.fillna({"Embarked": "S"})

# Categorize ages into meaningful groups
# Fill missing age values with -0.5 as a placeholder
train["Age"] = train["Age"].fillna(-0.5)
test["Age"] = test["Age"].fillna(-0.5)

# Define age bins and corresponding labels
age_bins = [-1, 0, 5, 12, 18, 24, 35, 60, np.inf]
age_labels = ['Unknown', 'Baby', 'Child', 'Teenager',
              'Student', 'Young Adult', 'Adult', 'Senior']

# Create 'AgeGroup' column by categorizing ages based on defined bins and labels
train['AgeGroup'] = pd.cut(train["Age"], bins=age_bins, labels=age_labels)
test['AgeGroup'] = pd.cut(test["Age"], bins=age_bins, labels=age_labels)


In [131]:
# Create a new column cabinbool indicating 
# if the cabin value is given '1' and not NaN '0'
train["CabinBool"] = (train["Cabin"].notnull().astype('int')) 
test["CabinBool"] = (test["Cabin"].notnull().astype('int')) 

# Delete the column 'Cabin' from test and train dataset 
train = train.drop(['Cabin'], axis=1) 
test = test.drop(['Cabin'], axis=1)


In [132]:
# Combine the train and test datasets into a list for easier manipulation
combine = [train, test]

# Extract titles from the 'Name' column in both datasets
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

# Display the frequency of titles based on gender in the training dataset
pd.crosstab(train['Title'], train['Sex'])

# Replace uncommon titles with more general categories
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(
        ['Lady', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Jonkheer', 'Dona'],
        'Rare'
    )
    dataset['Title'] = dataset['Title'].replace(
        ['Countess', 'Lady', 'Sir'], 'Royal'
    )
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

# Calculate the average survival rate for each title group in the training dataset
train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

# Convert titles to numerical values using a mapping dictionary
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Royal": 5, "Rare": 6}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)


In [133]:
mr_age = train[train["Title"] == 1]["AgeGroup"].mode() # Young Adult 
miss_age = train[train["Title"] == 2]["AgeGroup"].mode() # Student 
mrs_age = train[train["Title"] == 3]["AgeGroup"].mode() # Adult 
master_age = train[train["Title"] == 4]["AgeGroup"].mode() # Baby 
royal_age = train[train["Title"] == 5]["AgeGroup"].mode() # Adult 
rare_age = train[train["Title"] == 6]["AgeGroup"].mode() # Adult 

age_title_mapping = {1: "Young Adult", 2: "Student", 
					3: "Adult", 4: "Baby", 5: "Adult", 6: "Adult"} 

for x in range(len(train["AgeGroup"])): 
	if train["AgeGroup"][x] == "Unknown": 
		train["AgeGroup"][x] = age_title_mapping[train["Title"][x]] 

for x in range(len(test["AgeGroup"])): 
	if test["AgeGroup"][x] == "Unknown": 
		test["AgeGroup"][x] = age_title_mapping[test["Title"][x]] 

# map each Age value to a numerical value 
age_mapping = {'Baby': 1, 'Child': 2, 'Teenager': 3, 
			'Student': 4, 'Young Adult': 5, 'Adult': 6, 
			'Senior': 7} 
train['AgeGroup'] = train['AgeGroup'].map(age_mapping) 
test['AgeGroup'] = test['AgeGroup'].map(age_mapping) 

train.head() 

# dropping the Age feature for now, might change 
train = train.drop(['Age'], axis=1) 
test = test.drop(['Age'], axis=1) 


In [134]:
train = train.drop(['Name'], axis=1) 
test = test.drop(['Name'], axis=1) 


train = train.drop(['Ticket'], axis=1) 
test = test.drop(['Ticket'], axis=1) 

In [135]:
sex_mapping = {"male": 0, "female": 1} 
train['Sex'] = train['Sex'].map(sex_mapping) 
test['Sex'] = test['Sex'].map(sex_mapping) 

embarked_mapping = {"S": 1, "C": 2, "Q": 3} 
train['Embarked'] = train['Embarked'].map(embarked_mapping) 
test['Embarked'] = test['Embarked'].map(embarked_mapping) 


In [136]:
for x in range(len(test["Fare"])): 
	if pd.isnull(test["Fare"][x]): 
		pclass = test["Pclass"][x] # Pclass = 3 
		test["Fare"][x] = round( 
			train[train["Pclass"] == pclass]["Fare"].mean(), 4) 

# map Fare values into groups of numerical values 
train['FareBand'] = pd.qcut(train['Fare'], 4, 
							labels=[1, 2, 3, 4]) 
test['FareBand'] = pd.qcut(test['Fare'], 4, 
						labels=[1, 2, 3, 4]) 

# drop Fare values 
train = train.drop(['Fare'], axis=1) 
test = test.drop(['Fare'], axis=1) 




Modified Datasets

In [137]:
train.head(10)

,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Embarked,AgeGroup,CabinBool,Title,FareBand
0,1,0,3,0,1,0,1,4.0,0,1,1
1,2,1,1,1,1,0,2,6.0,1,3,4
2,3,1,3,1,0,0,1,5.0,0,2,2
3,4,1,1,1,1,0,1,5.0,1,3,4
4,5,0,3,0,0,0,1,5.0,0,1,2
5,6,0,3,0,0,0,3,5.0,0,1,2
6,7,0,1,0,0,0,1,6.0,1,1,4
7,8,0,3,0,3,1,1,1.0,0,4,3
8,9,1,3,1,0,2,1,5.0,0,3,2
9,10,1,2,1,1,0,2,3.0,0,3,3


In [138]:
test.head(10)

,PassengerId,Pclass,Sex,SibSp,Parch,Embarked,AgeGroup,CabinBool,Title,FareBand
0,1,3,0,1,0,1.0,4.0,0,1,1
1,2,1,1,1,0,2.0,6.0,1,3,4
2,3,3,1,0,0,1.0,5.0,0,2,2
3,4,1,1,1,0,1.0,5.0,1,3,4
4,5,3,0,0,0,1.0,5.0,0,1,2
5,6,3,0,0,0,3.0,5.0,0,1,2
6,7,1,0,0,0,1.0,6.0,1,1,4
7,8,3,0,3,1,1.0,1.0,0,4,3
8,9,3,1,0,2,1.0,5.0,0,3,2
9,10,2,1,1,0,2.0,3.0,0,3,3


Using Random forest Model

In [139]:
from sklearn.model_selection import train_test_split 

# Drop the Survived and PassengerId column from the trainset 
predictors = train.drop(['Survived', 'PassengerId'], axis=1) 
target = train["Survived"] 
x_train, x_val, y_train, y_val = train_test_split( 
	predictors, target, test_size=0.2, random_state=0) 


# Fit the RandomForestClassifier on the training data
randomforest = RandomForestClassifier()
randomforest.fit(x_train, y_train)

# Predict on the validation data
y_pred = randomforest.predict(x_val)

# Calculate the accuracy score
acc_randomforest = round(accuracy_score(y_pred, y_val) * 100, 2)
print(f"Validation Accuracy: {acc_randomforest}%")

# Separate numerical and categorical columns
numerical_cols = test.select_dtypes(include=['number']).columns
categorical_cols = test.select_dtypes(include=['object', 'category']).columns

# Fill missing values in numerical columns with the median
test[numerical_cols] = test[numerical_cols].fillna(test[numerical_cols].median())

# Fill missing values in categorical columns with the most frequent value (mode)
test[categorical_cols] = test[categorical_cols].fillna(test[categorical_cols].mode().iloc[0])

# Predict on the test data
ids = test['PassengerId']
predictions = randomforest.predict(test.drop('PassengerId', axis=1))

# Create the output DataFrame and save it to a CSV file
output = pd.DataFrame({'PassengerId': ids, 'Survived': predictions})
output.to_csv('resultfile.csv', index=False)




Validation Accuracy: 85.47%


In [140]:
result = pd.read_csv('resultfile.csv')
result.head(10)

,PassengerId,Survived
0,1,0
1,2,1
2,3,1
3,4,1
4,5,0
5,6,0
6,7,0
7,8,0
8,9,1
9,10,1


# Other Models for Comparission

Importing Models 

In [141]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

SVM Model

In [142]:
clf = SVC()
scoring = 'accuracy'
score = cross_val_score(clf, predictors, target, cv=k_fold, n_jobs=1, scoring=scoring)
print('SVM Model Accuracy = ',round(np.mean(score)*100,2))

SVM Model Accuracy =  83.16


Decision Tree Model

In [143]:
clf = DecisionTreeClassifier()
scoring = 'accuracy'
score = cross_val_score(clf, predictors, target, cv=k_fold, n_jobs=1, scoring=scoring)
print('Decision Tree Model Accuracy = ',round(np.mean(score)*100,2))

Decision Tree Model Accuracy =  79.35


Naive Bayes Model

In [144]:
clf = GaussianNB()
scoring = 'accuracy'
score = cross_val_score(clf, predictors, target, cv=k_fold, n_jobs=1, scoring=scoring)
print('Naive Bayes Model Accuracy = ',round(np.mean(score)*100,2))

Naive Bayes Model Accuracy =  78.22
